In [1]:
import pandas as pd
import numpy as np
from sklearn.datasets import load_boston
X, y = load_boston(return_X_y=True)
print(X.shape)
print(type(X))
print(y.shape)
print(type(y))

(506, 13)
<class 'numpy.ndarray'>
(506,)
<class 'numpy.ndarray'>


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function load_boston is deprecated; `load_boston` is deprecated in 1.0 and will be removed in 1.2.

    The Boston housing prices dataset has an ethical problem. You can refer to
    the documentation of this function for further details.

    The scikit-learn maintainers therefore strongly discourage the use of this
    dataset unless the purpose of the code is to study and educate about
    ethical issues in data science and machine learning.

    In this special case, you can fetch the dataset from the original
    source::

        import pandas as pd
        import numpy as np


        data_url = "http://lib.stat.cmu.edu/datasets/boston"
        raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
        data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
        target = raw_df.values[1::2, 2]

    Alternative datasets include the California housing dataset (i.e.

In [6]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

model = DecisionTreeRegressor()

# need to randomize X and y, but together
bostondf = pd.concat([pd.DataFrame(X), pd.Series(y)], axis=1)
bostondf.columns = [0,1,2,3,4,5,6,7,8,9,10,11,12,'y']
#bostondf.head()
bostondf = bostondf.drop(11, axis=1) # drop the 'B' column - no longer acceptable
bostondf = bostondf.sample(frac=1, random_state=1)
#bostondf.head()
X_rand=np.array(bostondf.loc[:, bostondf.columns != 'y'])
y_rand=np.array(bostondf['y'])

scores = cross_val_score(model, X_rand, y_rand, cv=5)
scores

# #alternatively: use a separate validation or hold-out set by using
# # X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.33)
# # and use X_val and y_val to validate the final model on
# # added bonus: train_test_split does the randomization for us!


array([0.66464405, 0.39262053, 0.79027057, 0.62580573, 0.84048271])

In [7]:
mean_score = scores.mean()
print(mean_score)

0.6627647179462091


# Comparing several models with CV

In [12]:
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
import numpy as np

model1 = DecisionTreeRegressor()
model2 = LinearRegression() # y = b0 + b1 * x1 + b2 * x2 +.....
model3 = KNeighborsRegressor() # weights = "uniform","distance"

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state = 1)

# data should really be scaled here
#scaler = StandardScaler()
#scaler.fit(X_train)
#X_train_scaled_np = scaler.transform(X_train)
#X_test_scaled_np  = scaler.transform(X_test)
#pd.DataFrame(X_train_scaled_np,)
#...
model_pipeline = [model1, model2, model3]
model_names = ['Decision Tree Regressor', 'Linear Regression', 'KNN']
scores = {}
for model, model_name in zip(model_pipeline, model_names):
    mean_score = np.mean(cross_val_score(model, X_train, y_train, cv=5))
    scores[model_name] = mean_score
print(scores)

# We can use the result to choose the best performing model

{'Decision Tree Regressor': 0.7719865569052276, 'Linear Regression': 0.7025123301096216, 'KNN': 0.4120954087230463}


In [13]:
list(zip(model_pipeline, model_names))

[(DecisionTreeRegressor(), 'Decision Tree Regressor'),
 (LinearRegression(), 'Linear Regression'),
 (KNeighborsRegressor(), 'KNN')]

In [14]:
val_scores = {}

for model, model_name in zip(model_pipeline,model_names):
    model.fit(X_train, y_train)
    val_scores[model_name] = model.score(X_val,y_val)
print(val_scores)

{'Decision Tree Regressor': 0.8193790788750704, 'Linear Regression': 0.7634174432138495, 'KNN': 0.5401612153026705}


# Activity 1

In this exercise we will go back to the customer churn data from the last lab (the dataset can also be found in the files_for_activities folder)

Implement cross validation along with logistic regression and decision tree classifier on the data

Create a pipeline as shown in the class example
You can use the following code to set up for this activity.
Finish the pipeline.

# Answer Activity 1

In [15]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [16]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
import warnings
warnings.filterwarnings('ignore')

churnData = pd.read_csv('/content/drive/MyDrive/CURR-v3.X-OCT2022/UNIT7/DAY3/7.07 Bootstrapping, bagging, RandomForest/Customer-Churn.csv')
churnData['TotalCharges']  = pd.to_numeric(churnData['TotalCharges'], errors='coerce')
churnData['TotalCharges'] = churnData['TotalCharges'].fillna(np.mean(churnData['TotalCharges']))

churnData['Churn']
churnData['Churn'].value_counts()

No     5174
Yes    1869
Name: Churn, dtype: int64

In [17]:
churn_no = churnData[['tenure', 'SeniorCitizen','MonthlyCharges', 'TotalCharges', 'Churn']][churnData['Churn']=='No']
churn_yes = churnData[['tenure', 'SeniorCitizen','MonthlyCharges', 'TotalCharges', 'Churn']][churnData['Churn']=='Yes']
churn_no_downsample = churn_no.sample(len(churn_yes))

In [18]:
print(churn_yes.shape)
print(churn_no_downsample.shape)

(1869, 5)
(1869, 5)


In [19]:
from sklearn.preprocessing import StandardScaler

churndf = pd.concat([churn_no_downsample, churn_yes], axis=0)
X = churndf[['tenure', 'SeniorCitizen','MonthlyCharges', 'TotalCharges']]
y = churndf['Churn']

scaler = StandardScaler()
scaler.fit(X)
X = scaler.transform(X)

In [20]:
X

array([[ 1.8556175 ,  2.06693357,  1.52649717,  2.74430765],
       [ 0.68155147, -0.48380849,  1.52131736,  1.34062523],
       [ 1.18472263, -0.48380849,  1.01887544,  1.58028906],
       ...,
       [-1.12147849,  2.06693357,  0.28679511, -0.90254113],
       [ 1.64596285, -0.48380849,  1.22606798,  2.25427263],
       [-0.9956857 ,  2.06693357,  0.24017679, -0.79553717]])

In [ ]:
#complete the code here

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.33)

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

logreg_model = LogisticRegression(random_state=0, solver='lbfgs', multi_class='ovr')
tree_model   = DecisionTreeClassifier()

import numpy as np
model_pipeline = [logreg_model, tree_model]
model_names = ['Logistic Regression', 'Decision Tree Classifier']
scores = {}

for model, model_name in zip(model_pipeline, model_names):
    mean_score = np.mean(cross_val_score(model, X_train, y_train, cv=10))
    scores[model_name] = mean_score
print(scores)

{'Logistic Regression': 0.7460143426294821, 'Decision Tree Classifier': 0.6641195219123506}


## end Activity 1

# Activity 2
- What are the advantages and disadvantages of using bootstrap method? Here is an [external resource](https://blog.paperspace.com/bagging-ensemble-methods/).
- What are the advantages and disadvantages of bagging?

# Answer Activity 2
### Bootstrap

- Advantages

  - Helps generate more samples from the original data. This is helpful when it is not possible to gather enough data from a process or the costs involved with taking new samples is high.
  - For sampling new data, it does not depend on the assumptions of underlying parametric distribution in the data.
  - Reduces variability in the data.

- Disadvantages

  - Depends on representative sample which could be good as well as bad. For eg.,if the original data has extreme values, bootstrap method will reduce the frequency of appearance of such values in the overall data. If those were values were actually outliers, then it is a good thing as we undermining the importance of such data points in the overall data at the end. However, if those data points are some rare important observations, then this could be bad.
  - The method does not work very well when the original sample size is too small.
  
---

### Bagging

- Advantages

  - Prevents over-fitting.
  - Different models are built independent of each other and equal weight is given to all the models.
  - The concept of "wisdom of crowds" which essentially means that the knowledge of a group of people is higher than knowledge of people independently.

- Disadvantages

  - Loss of interpretability; The final output is not interpretable like a decision where we can clearly see the decision spaces.
  - Computation complexity of the model increases.
  - It works well when the base classifier is good. If the base classifier is bad, then it can significantly decrease the performance.

## end Activity 2

# Activity 3

- Go through the documentation of random forests in `sklearn`, and get familiarized with the various parameters that can be used.
- There is another advanced concept called boosting. Conduct some elementary research on boosting and compare it with bagging.

# Answer Activity 3

The documentation: [here](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html).<br>
Boosting: detailed article [here](https://dataaspirant.com/ensemble-methods-bagging-vs-boosting-difference/)

# Random Forest

In [21]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split


numerical = pd.read_csv('/content/drive/MyDrive/CURR-v3.X-OCT2022/UNIT7/DAY1/7.01, 7.02 Data Cleaning and Feature selection review/numerical7_02.csv')
categorical = pd.read_csv('/content/drive/MyDrive/CURR-v3.X-OCT2022/UNIT7/DAY1/7.01, 7.02 Data Cleaning and Feature selection review/categorical7_02.csv')
targets = pd.read_csv('/content/drive/MyDrive/CURR-v3.X-OCT2022/UNIT7/DAY1/7.01, 7.02 Data Cleaning and Feature selection review/target7_02.csv')
data = pd.concat([numerical, categorical, targets], axis = 1)
data['TARGET_B'].value_counts()

0    90569
1     4843
Name: TARGET_B, dtype: int64

In [22]:
data.drop(columns=['Unnamed: 0'], axis = 1, inplace = True)

In [23]:
data.head()

,ODATEDW,TCODE,DOB,AGE,INCOME,WEALTH1,HIT,MALEMILI,MALEVET,VIETVETS,WWIIVETS,LOCALGOV,STATEGOV,FEDGOV,WEALTH2,POP901,POP902,POP903,POP90C1,POP90C2,POP90C3,POP90C4,POP90C5,ETH1,ETH2,ETH3,ETH4,ETH5,ETH6,ETH7,ETH8,ETH9,ETH10,ETH11,ETH12,ETH13,ETH14,ETH15,ETH16,AGE901,AGE902,AGE903,AGE904,AGE905,AGE906,AGE907,CHIL1,CHIL2,CHIL3,AGEC1,AGEC2,AGEC3,AGEC4,AGEC5,AGEC6,AGEC7,CHILC1,CHILC2,CHILC3,CHILC4,CHILC5,HHAGE1,HHAGE2,HHAGE3,HHN1,HHN2,HHN3,HHN4,HHN5,HHN6,MARR1,MARR2,MARR3,MARR4,HHP1,HHP2,DW1,DW2,DW3,DW4,DW5,DW6,DW7,DW8,DW9,HV1,HV2,HV3,HV4,HU1,HU2,HU3,HU4,HU5,HHD1,HHD2,HHD3,HHD4,HHD5,HHD6,HHD7,HHD8,HHD9,HHD10,HHD11,HHD12,ETHC1,ETHC2,ETHC3,ETHC4,ETHC5,ETHC6,HVP1,HVP2,HVP3,HVP4,HVP5,HVP6,HUR1,HUR2,RHP1,RHP2,RHP3,RHP4,HUPA1,HUPA2,HUPA3,HUPA4,HUPA5,HUPA6,HUPA7,RP1,RP2,RP3,RP4,MSA,ADI,DMA,IC1,IC2,IC3,IC4,IC5,IC6,IC7,IC8,IC9,IC10,IC11,IC12,IC13,IC14,IC15,IC16,IC17,IC18,IC19,IC20,IC21,IC22,IC23,HHAS1,HHAS2,HHAS3,HHAS4,MC1,MC2,MC3,TPE1,TPE2,TPE3,TPE4,TPE5,TPE6,TPE7,TPE8,TPE9,PEC1,PEC2,TPE10,TPE11,TPE12,TPE13,LFC1,LFC2,LFC3,LFC4,LFC5,LFC6,LFC7,LFC8,LFC9,LFC10,OCC1,OCC2,OCC3,OCC4,OCC5,OCC6,OCC7,OCC8,OCC9,OCC10,OCC11,OCC12,OCC13,EIC1,EIC2,EIC3,EIC4,EIC5,EIC6,EIC7,EIC8,EIC9,EIC10,EIC11,EIC12,EIC13,EIC14,EIC15,EIC16,OEDC1,OEDC2,OEDC3,OEDC4,OEDC5,OEDC6,OEDC7,EC1,EC2,EC3,EC4,EC5,EC6,EC7,EC8,SEC1,SEC2,SEC3,SEC4,SEC5,AFC1,AFC2,AFC3,AFC4,AFC5,AFC6,VC1,VC2,VC3,VC4,ANC1,ANC2,ANC3,ANC4,ANC5,ANC6,ANC7,ANC8,ANC9,ANC10,ANC11,ANC12,ANC13,ANC14,ANC15,POBC1,POBC2,LSC1,LSC2,LSC3,LSC4,VOC1,VOC2,VOC3,HC1,HC2,HC3,HC4,HC5,HC6,HC7,HC8,HC9,HC10,HC11,HC12,HC13,HC14,HC15,HC16,HC17,HC18,HC19,HC20,HC21,MHUC1,MHUC2,AC1,AC2,CARDPROM,MAXADATE,NUMPROM,CARDPM12,NUMPRM12,RAMNTALL,NGIFTALL,CARDGIFT,MINRAMNT,MINRDATE,MAXRAMNT,MAXRDATE,LASTGIFT,LASTDATE,FISTDATE,NEXTDATE,TIMELAG,AVGGIFT,CONTROLN,HPHONE_D,RFA_2F,CLUSTER2,OSOURCE,STATE,ZIP,MAILCODE,NOEXCH,MDMAUD,CLUSTER,HOMEOWNR,GENDER,DATASRCE,RFA_2R,RFA_2A,GEOCODE2,DOMAIN_A,DOMAIN_B,TARGET_B,TARGET_D
0,9401.0,1.0,5202.0,46.0,6.0,9.0,16.0,0.0,15.0,55.0,11.0,6.0,2.0,1.0,9.0,3611.0,940.0,998.0,99.0,0.0,0.0,50.0,50.0,67.0,0.0,0.0,31.0,6.0,4.0,2.0,6.0,4.0,14.0,0.0,0.0,2.0,0.0,1.0,4.0,34.0,41.0,43.0,32.0,42.0,45.0,32.0,33.0,46.0,21.0,13.0,14.0,33.0,23.0,10.0,4.0,2.0,11.0,16.0,36.0,22.0,15.0,12.0,1.0,5.0,4.0,21.0,75.0,55.0,23.0,9.0,69.0,4.0,3.0,24.0,317.0,360.0,99.0,99.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5468.0,5218.0,12.0,10.0,96.0,4.0,97.0,3.0,9.0,59.0,94.0,88.0,55.0,95.0,5.0,4.0,1.0,3.0,5.0,4.0,2.0,18.0,44.0,5.0,0.0,0.0,0.0,97.0,98.0,98.0,98.0,99.0,94.0,0.0,83.0,76.0,73.0,21.0,5.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,91.0,91.0,91.0,94.0,4480.0,13.0,803.0,1088.0,1096.0,1026.0,1037.0,36175.0,2.0,6.0,2.0,5.0,15.0,14.0,13.0,10.0,33.0,2.0,5.0,2.0,5.0,15.0,14.0,14.0,10.0,32.0,6.0,2.0,66.0,3.0,56.0,44.0,9.0,80.0,14.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0,2.0,24.0,32.0,12.0,71.0,70.0,83.0,58.0,81.0,57.0,64.0,57.0,99.0,99.0,0.0,22.0,24.0,4.0,21.0,13.0,2.0,1.0,6.0,0.0,4.0,1.0,0.0,3.0,1.0,0.0,6.0,13.0,1.0,2.0,8.0,18.0,11.0,4.0,3.0,4.0,10.0,7.0,11.0,1.0,6.0,2.0,1.0,16.0,69.0,5.0,2.0,160.0,5.0,5.0,12.0,21.0,7.0,30.0,20.0,14.0,24.0,4.0,24.0,10.0,0.0,0.0,0.0,8.0,15.0,0.0,55.0,10.0,11.0,0.0,0.0,2.0,0.0,3.0,1.0,1.0,2.0,3.0,1.0,1.0,0.0,3.0,0.0,0.0,0.0,42.0,39.0,50.0,7.0,27.0,16.0,99.0,92.0,53.0,5.0,10.0,2.0,26.0,56.0,97.0,99.0,0.0,0.0,0.0,96.0,0.0,4.0,0.0,0.0,0.0,99.0,0.0,99.0,99.0,99.0,20.0,4.0,6.0,5.0,12.0,9702.0,32.0,6.0,13.0,47.0,3.0,1.0,10.0,9310.0,25.0,9512.0,25.0,9512.0,9310.0,9504.0,18.0,15.666667,148535.0,0.0,2.0,1.0,BOA,CA,91326,A,0,XXXX,14.0,H,M,3.0,L,G,A,S,1.0,0,0.0
1,9001.0,1.0,0.0,NaN,3.0,1.0,2.0,0.0,20.0,29.0,33.0,6.0,8.0,1.0,1.0,7001.0,2040.0,2669.0,0.0,2.0,98.0,49.0,51.0,96.0,2.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,35.0,43.0,46.0,37.0,45.0,49.0,23.0,35.0,40.0,25.0,13.0,20.0,19.0,16.0,13.0,10.0,8.0,15.0,14.0,30.0,22.0,19.0,25.0,10.0,23.0,21.0,35.0,44.0,22.0,6.0,2.0,63.0,9.0,9.0,19.0,183.0,254.0,69.0,69.0,1.0,6.0,5.0,3.0,3.0,3.0,0.0,497.0,546.0,2.0,1.0,78.0,22.0,93.0,7.0,18.0,36.0,76.0,65.0,30.0,86.0,14.0,7.0,2.0,5.0,11.0,17.0,3.0,17.0,6

We still had lots of NA's even though the percentage was small.

In [24]:
data.isna().sum()

ODATEDW     44794
TCODE       44794
DOB         44794
AGE         51436
INCOME      44794
            ...  
GEOCODE2    44794
DOMAIN_A    44794
DOMAIN_B    44794
TARGET_B        0
TARGET_D        0
Length: 340, dtype: int64

In [ ]:
# To simplify, let's drop any row with an NA on any column

In [25]:
data.dropna(axis= 0,inplace=True)

In [26]:
data.head()

,ODATEDW,TCODE,DOB,AGE,INCOME,WEALTH1,HIT,MALEMILI,MALEVET,VIETVETS,WWIIVETS,LOCALGOV,STATEGOV,FEDGOV,WEALTH2,POP901,POP902,POP903,POP90C1,POP90C2,POP90C3,POP90C4,POP90C5,ETH1,ETH2,ETH3,ETH4,ETH5,ETH6,ETH7,ETH8,ETH9,ETH10,ETH11,ETH12,ETH13,ETH14,ETH15,ETH16,AGE901,AGE902,AGE903,AGE904,AGE905,AGE906,AGE907,CHIL1,CHIL2,CHIL3,AGEC1,AGEC2,AGEC3,AGEC4,AGEC5,AGEC6,AGEC7,CHILC1,CHILC2,CHILC3,CHILC4,CHILC5,HHAGE1,HHAGE2,HHAGE3,HHN1,HHN2,HHN3,HHN4,HHN5,HHN6,MARR1,MARR2,MARR3,MARR4,HHP1,HHP2,DW1,DW2,DW3,DW4,DW5,DW6,DW7,DW8,DW9,HV1,HV2,HV3,HV4,HU1,HU2,HU3,HU4,HU5,HHD1,HHD2,HHD3,HHD4,HHD5,HHD6,HHD7,HHD8,HHD9,HHD10,HHD11,HHD12,ETHC1,ETHC2,ETHC3,ETHC4,ETHC5,ETHC6,HVP1,HVP2,HVP3,HVP4,HVP5,HVP6,HUR1,HUR2,RHP1,RHP2,RHP3,RHP4,HUPA1,HUPA2,HUPA3,HUPA4,HUPA5,HUPA6,HUPA7,RP1,RP2,RP3,RP4,MSA,ADI,DMA,IC1,IC2,IC3,IC4,IC5,IC6,IC7,IC8,IC9,IC10,IC11,IC12,IC13,IC14,IC15,IC16,IC17,IC18,IC19,IC20,IC21,IC22,IC23,HHAS1,HHAS2,HHAS3,HHAS4,MC1,MC2,MC3,TPE1,TPE2,TPE3,TPE4,TPE5,TPE6,TPE7,TPE8,TPE9,PEC1,PEC2,TPE10,TPE11,TPE12,TPE13,LFC1,LFC2,LFC3,LFC4,LFC5,LFC6,LFC7,LFC8,LFC9,LFC10,OCC1,OCC2,OCC3,OCC4,OCC5,OCC6,OCC7,OCC8,OCC9,OCC10,OCC11,OCC12,OCC13,EIC1,EIC2,EIC3,EIC4,EIC5,EIC6,EIC7,EIC8,EIC9,EIC10,EIC11,EIC12,EIC13,EIC14,EIC15,EIC16,OEDC1,OEDC2,OEDC3,OEDC4,OEDC5,OEDC6,OEDC7,EC1,EC2,EC3,EC4,EC5,EC6,EC7,EC8,SEC1,SEC2,SEC3,SEC4,SEC5,AFC1,AFC2,AFC3,AFC4,AFC5,AFC6,VC1,VC2,VC3,VC4,ANC1,ANC2,ANC3,ANC4,ANC5,ANC6,ANC7,ANC8,ANC9,ANC10,ANC11,ANC12,ANC13,ANC14,ANC15,POBC1,POBC2,LSC1,LSC2,LSC3,LSC4,VOC1,VOC2,VOC3,HC1,HC2,HC3,HC4,HC5,HC6,HC7,HC8,HC9,HC10,HC11,HC12,HC13,HC14,HC15,HC16,HC17,HC18,HC19,HC20,HC21,MHUC1,MHUC2,AC1,AC2,CARDPROM,MAXADATE,NUMPROM,CARDPM12,NUMPRM12,RAMNTALL,NGIFTALL,CARDGIFT,MINRAMNT,MINRDATE,MAXRAMNT,MAXRDATE,LASTGIFT,LASTDATE,FISTDATE,NEXTDATE,TIMELAG,AVGGIFT,CONTROLN,HPHONE_D,RFA_2F,CLUSTER2,OSOURCE,STATE,ZIP,MAILCODE,NOEXCH,MDMAUD,CLUSTER,HOMEOWNR,GENDER,DATASRCE,RFA_2R,RFA_2A,GEOCODE2,DOMAIN_A,DOMAIN_B,TARGET_B,TARGET_D
0,9401.0,1.0,5202.0,46.0,6.0,9.0,16.0,0.0,15.0,55.0,11.0,6.0,2.0,1.0,9.0,3611.0,940.0,998.0,99.0,0.0,0.0,50.0,50.0,67.0,0.0,0.0,31.0,6.0,4.0,2.0,6.0,4.0,14.0,0.0,0.0,2.0,0.0,1.0,4.0,34.0,41.0,43.0,32.0,42.0,45.0,32.0,33.0,46.0,21.0,13.0,14.0,33.0,23.0,10.0,4.0,2.0,11.0,16.0,36.0,22.0,15.0,12.0,1.0,5.0,4.0,21.0,75.0,55.0,23.0,9.0,69.0,4.0,3.0,24.0,317.0,360.0,99.0,99.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5468.0,5218.0,12.0,10.0,96.0,4.0,97.0,3.0,9.0,59.0,94.0,88.0,55.0,95.0,5.0,4.0,1.0,3.0,5.0,4.0,2.0,18.0,44.0,5.0,0.0,0.0,0.0,97.0,98.0,98.0,98.0,99.0,94.0,0.0,83.0,76.0,73.0,21.0,5.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,91.0,91.0,91.0,94.0,4480.0,13.0,803.0,1088.0,1096.0,1026.0,1037.0,36175.0,2.0,6.0,2.0,5.0,15.0,14.0,13.0,10.0,33.0,2.0,5.0,2.0,5.0,15.0,14.0,14.0,10.0,32.0,6.0,2.0,66.0,3.0,56.0,44.0,9.0,80.0,14.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0,2.0,24.0,32.0,12.0,71.0,70.0,83.0,58.0,81.0,57.0,64.0,57.0,99.0,99.0,0.0,22.0,24.0,4.0,21.0,13.0,2.0,1.0,6.0,0.0,4.0,1.0,0.0,3.0,1.0,0.0,6.0,13.0,1.0,2.0,8.0,18.0,11.0,4.0,3.0,4.0,10.0,7.0,11.0,1.0,6.0,2.0,1.0,16.0,69.0,5.0,2.0,160.0,5.0,5.0,12.0,21.0,7.0,30.0,20.0,14.0,24.0,4.0,24.0,10.0,0.0,0.0,0.0,8.0,15.0,0.0,55.0,10.0,11.0,0.0,0.0,2.0,0.0,3.0,1.0,1.0,2.0,3.0,1.0,1.0,0.0,3.0,0.0,0.0,0.0,42.0,39.0,50.0,7.0,27.0,16.0,99.0,92.0,53.0,5.0,10.0,2.0,26.0,56.0,97.0,99.0,0.0,0.0,0.0,96.0,0.0,4.0,0.0,0.0,0.0,99.0,0.0,99.0,99.0,99.0,20.0,4.0,6.0,5.0,12.0,9702.0,32.0,6.0,13.0,47.0,3.0,1.0,10.0,9310.0,25.0,9512.0,25.0,9512.0,9310.0,9504.0,18.0,15.666667,148535.0,0.0,2.0,1.0,BOA,CA,91326,A,0,XXXX,14.0,H,M,3.0,L,G,A,S,1.0,0,0.0
2,8701.0,0.0,2801.0,70.0,1.0,4.0,2.0,0.0,23.0,14.0,31.0,3.0,0.0,3.0,0.0,640.0,160.0,219.0,0.0,8.0,92.0,54.0,46.0,61.0,0.0,0.0,11.0,32.0,6.0,2.0,0.0,0.0,0.0,0.0,0.0,31.0,0.0,0.0,1.0,32.0,40.0,44.0,34.0,43.0,47.0,25.0,45.0,35.0,20.0,15.0,25.0,17.0,17.0,12.0,7.0,7.0,20.0,17.0,30.0,14.0,19.0,25.0,11.0,23.0,23.0,27.0,50.0,30.0,15.0,8.0,63.0,9.0,6.0,23.0,199.0,283.0,85.0,83.0,3.0,4.0,1.0,0.0,2.0,0.0,2.0,1000.0,1263.0,2.0,1.0,48.0,52.0,93.0,7.0,6.0,36.0,73.0,61.0,30.0,84.0,16.0,6.0,3.0,3.0,21.0,12.0,4.0,1

In [27]:
data.reset_index(drop=True, inplace=True)

In [28]:
data.shape

(25251, 340)

In [29]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

y = data['TARGET_B']
X = data.drop(['TARGET_B'], axis = 1)

numericalX = X.select_dtypes(np.number)
categoricalX = X.select_dtypes(np.object)

display(categoricalX.head())

for col in categoricalX.columns:
    print(col, categoricalX[col].nunique())

print()
# We're going to drop columns "OSOURCE" and "ZIP" because they too many levels.
categoricalX.drop(columns = ['OSOURCE','ZIP'], axis = 1, inplace = True)
# we OneHotEncode the categoricals so we can use the same dataset to perform a regression later (in the lab).
# even though it is not needed for a DecisionTree or RandomForest model

encoder = OneHotEncoder(drop='first').fit(categoricalX)
encoded_categorical = encoder.transform(categoricalX).toarray()
encoded_categorical = pd.DataFrame(encoded_categorical)
X = pd.concat([numericalX, encoded_categorical], axis = 1)

# Note: we need to do train/test split before downsampling, and then only downsample the training set - Why?

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)


,OSOURCE,STATE,ZIP,MAILCODE,NOEXCH,MDMAUD,HOMEOWNR,GENDER,RFA_2R,RFA_2A,GEOCODE2,DOMAIN_A
0,BOA,CA,91326,A,0,XXXX,H,M,L,G,A,S
1,BRY,CA,95953,A,0,XXXX,U,F,L,E,C,R
2,DRK,IN,46755,A,0,XXXX,H,F,L,E,D,T
3,ENQ,other,56475,A,0,XXXX,H,F,L,D,D,R
4,USB,other,84720,A,0,XXXX,H,M,L,F,D,T


OSOURCE 580
STATE 12
ZIP 9098
MAILCODE 2
NOEXCH 3
MDMAUD 17
HOMEOWNR 2
GENDER 3
RFA_2R 1
RFA_2A 4
GEOCODE2 4
DOMAIN_A 5



In [30]:
data['TARGET_B'].value_counts()

0    23987
1     1264
Name: TARGET_B, dtype: int64

In [31]:
# for downsampling we need to temporarily concat X_train and y_train
trainset = pd.concat([X_train, y_train], axis=1)

# Manual way to downsample category 0:
category_0 = trainset[trainset['TARGET_B']==0].sample(len(trainset[trainset['TARGET_B']==1]))
print(category_0.shape)

category_1 = trainset[trainset['TARGET_B']== 1 ]
trainset_new = pd.concat([category_0, category_1], axis = 0)
trainset_new = trainset_new.sample(frac =1) #randomize the rows
X_train = trainset_new.drop(['TARGET_B'], axis=1)
y_train = trainset_new['TARGET_B']
#data = data.reset_index(drop=True)
print(X_train.shape)

(1026, 371)
(2052, 370)


In [32]:
X_train = pd.DataFrame(X_train)
X_test = pd.DataFrame(X_test)

y_train_regression = X_train['TARGET_D']
y_test_regression = X_test['TARGET_D']

# Now we can remove the column target d from the set of features
X_train = X_train.drop(['TARGET_D'], axis = 1)
X_test = X_test.drop(['TARGET_D'], axis = 1)

In [33]:
y_test

16131    0
14503    0
19703    0
7849     0
13285    1
        ..
17316    0
249      0
18510    0
7449     0
7485     0
Name: TARGET_B, Length: 5051, dtype: int64

In [34]:
y_test_regression

16131    0.0
14503    0.0
19703    0.0
7849     0.0
13285    6.0
        ... 
17316    0.0
249      0.0
18510    0.0
7449     0.0
7485     0.0
Name: TARGET_D, Length: 5051, dtype: float64

In [36]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix

clf = RandomForestClassifier(max_depth=5,
                             min_samples_split=20,
                             min_samples_leaf =20,
                             max_samples=0.8,
                             random_state = 42)
clf.fit(X_train, y_train)

print("The Accuracy for the Random Forest in the TRAIN set is {:.2f}".format(clf.score(X_train, y_train)))
print("The Accuracy for the Random Forest in the TEST  set is {:.2f}".format(clf.score(X_test, y_test)))

y_pred = clf.predict(X_test)
display(y_test.value_counts())
display(confusion_matrix(y_test, y_pred))

The Accuracy for the Random Forest in the TRAIN set is 0.77
The Accuracy for the Random Forest in the TEST  set is 0.52


0    4813
1     238
Name: TARGET_B, dtype: int64

array([[2526, 2287],
       [ 126,  112]])

In [38]:
# For cross validation
from sklearn.model_selection import cross_val_score

clf = RandomForestClassifier(max_depth=5,
                             min_samples_split=20,
                             min_samples_leaf =20,
                             max_samples=0.8)

cross_val_scores = cross_val_score(clf, X_train, y_train, cv=10)

print("The mean Accuracy of the folds was {:.2f}".format(np.mean(cross_val_scores)))

The mean Accuracy of the folds was 0.51


In [39]:
cross_val_scores

array([0.44660194, 0.5631068 , 0.50243902, 0.51219512, 0.53658537,
       0.43414634, 0.52195122, 0.52195122, 0.52682927, 0.53658537])